In [126]:
file_name = '../../storage/app/uploads/uGAY4nnaxlVNY467IvfS4GOL1CTfbv13IP4axaT5.bin'


In [118]:
import mysql.connector
import os
import pandas as pd


# HOST = 

# Connect to the database
connection = mysql.connector.connect(
    host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
    user="admin",
    passwd="cujwiq-suqhu2-bycpuB",
    database="web-app"
)

# Create a cursor
cursor = connection.cursor()

# Execute a query
cursor.execute("SELECT * FROM models")

# Fetch the results
results = cursor.fetchall()

# Print the results
for result in results:
    print(result)
    
# Close the connection
connection.close()

def execute_query(query):
    connection = mysql.connector.connect(
        host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
        user="admin",
        passwd="cujwiq-suqhu2-bycpuB",
        database="web-app"
    )
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    connection.close()
    return results
    


(1, 'hankam', 'model hankam', '', 0, datetime.datetime(2024, 6, 23, 2, 30, 59), datetime.datetime(2024, 8, 3, 6, 5, 34), None)
(2, 'Model', 'k kb', 'uploads/cVAfVdp8aKiEyFMFuedSMGkcWZ6txkspfLt7LnXu.txt', 0, datetime.datetime(2024, 7, 3, 14, 56, 2), datetime.datetime(2024, 8, 3, 5, 49, 4), None)
(3, 'Model 2', 'Model 2', 'uploads/bSNnVxitl8HvD3VcAXyR3ek6jfiJveGJgFO4EXoL.txt', 0, datetime.datetime(2024, 7, 3, 15, 12, 41), datetime.datetime(2024, 8, 3, 5, 49, 4), None)
(4, 'MODEL', '', None, 0, datetime.datetime(2024, 7, 4, 11, 19, 26), datetime.datetime(2024, 7, 4, 11, 19, 26), None)
(5, 'asd', 'asda', 'uploads/nujT70ak7rLU5k8IpwbBBYHh3KNugKMzcLMcim0Z.bin', 0, datetime.datetime(2024, 7, 4, 11, 28, 35), datetime.datetime(2024, 8, 3, 5, 49, 4), None)
(6, 'tes 1', 'tes 1tes 1', 'uploads/25oe2FGbrIIIwtelNJicj23q5Ue94ENNzdNkoM6V.txt', 0, datetime.datetime(2024, 8, 3, 1, 55, 58), datetime.datetime(2024, 8, 3, 5, 49, 4), 'imageModels/q8wMrm0gz7Sk9I1am4B5BWBf7zivp5g520uyeJ3m.png')
(7, 'MODEL TES

In [14]:
import pandas as pd
import json
import pyodbc
pyodbc.drivers()

['ODBC Driver 17 for SQL Server',
 'ODBC Driver 18 for SQL Server',
 'MySQL-NATUNA',
 'ODBC Drivers',
 'ODBC Connection Pooling']

# Read Model

In [13]:
import json
import pandas as pd

In [14]:
file_name = 'model.mdl'
# file_name = 'Most Likely.mdl'

In [117]:
content_splitter

'********************************************************'

In [127]:
def load_model(file_name, model_id):
    # Open the .mdl file in read mode
    with open(file_name, 'r', encoding='utf-8') as file:
        # Read the content of the file
        mdl_content = file.read()
    # Split the content of the file by the newline character


    mdl_content = mdl_content.split('********************************************************')[0].replace('{UTF-8}', '\n').replace('\n','').replace('\t', '')
    variables = mdl_content.split('|')
    
    result = []
    for item in variables[:-1]:
        result.append(item.split('\t'))
        
    
    final = []
    for item in result:
        item = item[0]
        temp = {}
        try:
            # print(item)
            temp_list = item.split('~')        
            # print(temp_list)
            
            # temp_list = temp_list[1].split('~')
            temp['name'] = temp_list[0].split('=')[0]
            # print(len(temp_list[0].split('=')[0]))
            temp['value'] = '='.join(temp_list[0].split('=')[1:]).strip()
            temp['level'] = temp_list[-2]
            temp['unit'] = temp_list[-1]
            
            # temp_list = temp_list[0].split('=')[0]
            
        except Exception as e:
            print('error')
            print(str(e))
            print(item)
            break
        final.append(temp)
    
    df = pd.read_json(json.dumps(final))
    df = df.fillna('NULL')
    df = df.replace('', 'NULL')
    df['model_id'] = model_id
    
    return df
        
    

In [128]:
model_id = 1
data = load_model(file_name, model_id)
data

,name,value,level,unit,model_id
0,Level of Threat,National Sea Threat Risk/North Natuna Defense ...,NULL,NULL,1
1,Defense Score,Naval Defense Posture/North Natuna Defense and...,NULL,NULL,1
2,Foreign Policy,"(RANDOM UNIFORM( 1 , 5 , 1 ))*20",NULL,NULL,1
3,Misuses of AIS and Positioning Data Probability,"RANDOM UNIFORM( 1 , 5 , 1 )",NULL,NULL,1
4,Combined Deploy,"(RANDOM UNIFORM( 1 , 5 , 1 ))*20",NULL,NULL,1
...,...,...,...,...,...
94,Number of Non Military Threats,6,NULL,NULL,1
95,Social Media Level of Threats,Social Media Probability+Social Media Severity,NULL,NULL,1
96,Risk Level Maximum Score,9,NULL,NULL,1
97,Proxy Level of Threats,Proxy Probability+Proxy Severity,NULL,NULL,1


In [23]:
def load_raw(file_name):
    # Open the .mdl file in read mode
    with open(file_name, 'r', encoding='utf-8') as file:
        # Read the content of the file
        mdl_content = file.read()
    return mdl_content

In [34]:
raw = load_raw(file_name).split('********************************************************')[-1].split('|')[1]
raw

'\n\nFINAL TIME  = 120\n\t~\tMonth\n\t~\tThe final time for the simulation.\n\t'

In [42]:
def get_final_time(model):
    model = load_raw(file_name).split('********************************************************')[-1].split('|')[1]
    model = model.split('\n')
    final_time = 0
    
    for m in model:
        if 'FINAL TIME' in m:
            final_time = m.split('=')[-1].strip()  
    
    # return model
    return int(final_time)

get_final_time(file_name)

120

In [116]:
def get_sfd_name(file_name):
    model = load_raw(file_name).split('********************************************************')[-1].split('*')
    
    # last model modify to split "///---\\\"
    temp =  model[-1].split('///---\\\n')[0]
    
    #change the last model to temp
    model[-1] = temp
    number_of_sfd = 0
    sfds = []
    
    for m in model:
        if 'SFD' in m:
            number_of_sfd += 1
            sfd_name = m.split('\n')[0].split('[SFD]')[-1].strip()
            sfds.append(sfd_name)
            
    return sfds
        
get_sfd_name(file_name)

['Naval Strength',
 'Naval Deployment',
 'Naval Capabilities',
 'Naval Defense Posture']

In [112]:
def get_sfd(model, model_id):
    model = load_raw(file_name).split('********************************************************')[-1].split('*')
    
    # last model modify to split "///---\\\"
    temp =  model[-1].split('///---\\\n')[0]
    
    #change the last model to temp
    model[-1] = temp
    number_of_sfd = 0
    all_sfd_variables = []
    
    for m in model:
        if 'SFD' in m:
            number_of_sfd += 1
            sfd_name = m.split('\n')[0].split('[SFD]')[-1].strip()
            sfd_variables = []
            variables = m.split('\n')

            for variable in variables:
                # if startwith 10 get the variable name
                try:
                    if variable.startswith('10'):
                        # print(variable_name)
                        variable_name = variable.split(',')[2].strip()
                        #  if variable not string then skip
                        if not variable_name[0].isalpha():
                            continue
                        sfd_variables.append(variable_name)
                except Exception as e:
                    pass
            
            for variable in sfd_variables:
                all_sfd_variables.append({
                    'sfd_name': sfd_name,
                    'variable': variable.strip(),
                    'model_id': model_id
                })

    return all_sfd_variables

get_sfd(file_name, model_id)

[{'sfd_name': 'Naval Strength',
  'variable': 'Combat Vehicle Score',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Marine Ability Score',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Marine Forces Strength',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Aircraft Strength',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Warships Strength',
  'model_id': 1},
 {'sfd_name': 'Naval Strength', 'variable': 'Naval Strength', 'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Fighters Aircraft Score',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Enablers Aircraft',
  'model_id': 1},
 {'sfd_name': 'Naval Strength',
  'variable': 'Marine Personel Forces',
  'model_id': 1},
 {'sfd_name': 'Naval Strength', 'variable': 'Harbour', 'model_id': 1},
 {'sfd_name': 'Naval Strength', 'variable': 'Rest', 'model_id': 1},
 {'sfd_name': 'Naval Strength', 'variable': 'Replenishment', 'model_id': 1},
 {'s

In [113]:
data_sfd = pd.read_json(json.dumps(get_sfd(file_name, model_id)))
data_sfd

,sfd_name,variable,model_id
0,Naval Strength,Combat Vehicle Score,1
1,Naval Strength,Marine Ability Score,1
2,Naval Strength,Marine Forces Strength,1
3,Naval Strength,Aircraft Strength,1
4,Naval Strength,Warships Strength,1
5,Naval Strength,Naval Strength,1
6,Naval Strength,Fighters Aircraft Score,1
7,Naval Strength,Enablers Aircraft,1
8,Naval Strength,Marine Personel Forces,1
9,Naval Strength,Harbour,1


In [34]:
query = f"INSERT INTO variables (name, value, level, unit,  model_id) VALUES {', '.join([str(tuple(row)) for row in final_data.values])}"
query

try:
    connection = mysql.connector.connect(
        host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
        user="admin",
        passwd="cujwiq-suqhu2-bycpuB",
        database="web-app"
    )
    cursor = connection.cursor()
    cursor.execute(query)
    connection.commit()
    connection.close()
    print('success')
except Exception as e:
    print('error')
    print(str(e))
    connection.close()
    print('error')
    

success


In [39]:
# Open the .mdl file in read mode
with open(file_name, 'r', encoding='utf-8') as file:
    # Read the content of the file
    mdl_content = file.read()

In [41]:
# read view variables
mdl_content.split('VIEW-Naval Strength')[1].split('\n')


['',
 '$-1--1--1,0,|12||-1--1--1|-1--1--1|-1--1--1|-1--1--1|-1--1--1|96,96,121,0',
 '10,1,Number of Combat Vehicle,385,593,60,24,3,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,2,Marine Ability Score,248,185,75,30,8,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,3,Marine Forces Strength,385,293,58,58,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '1,4,2,3,1,0,0,0,0,192,0,-1--1--1,,1|(342,215)|',
 '10,5,Aircraft Strength,518,49,55,55,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '10,6,Warships Strength,1002,269,55,55,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '10,7,Fighters Aircraft Score,348,-68,56,56,2,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,8,Enablers Aircraft,347,123,75,30,8,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '1,9,7,5,1,0,0,0,0,192,0,-1--1--1,,1|(428,-55)|',
 '1,10,8,5,1,0,0,0,0,192,0,-1--1--1,,1|(445,118)|',
 '10,11,Marine Personel Forces,-71,403,75,30,3,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,12,Harbour,736,362,49,49,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',

# Generate Model